# CPSC 330 Lecture 5

#### Lecture plan

- 👋
- **Turn on recording**
- Announcements
- True/False questions from last time (10 min)
- Pipelines motivation (10 min)
- Pipelines (15 min)
- Break (5-10 min)
- Hyperparameter optimization: grid search and random search (30 min)

Bonus:

- Bayesian hyperparameter optimization (10 min)


In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

## Announcements

- hw2 solutions posted.
- hw3 posted, due Monday at 11:59pm.
  - You can work with a partner, see instructions [here](https://github.com/UBC-CS/cpsc330/blob/master/docs/homework_instructions.md#partners).

## True/False questions from last time (10 min)

https://piazza.com/class/kb2e6nwu3uj23?cid=188

## Pipelines motivation (10 min)

Returning to our dataset of the week, which is IMDB movie reviews:

In [2]:
imdb_df = pd.read_csv('data/imdb_master.csv', index_col=0, encoding="ISO-8859-1")
imdb_df = imdb_df[imdb_df['label'].str.startswith(('pos','neg'))]
imdb_df = imdb_df.sample(frac=0.2, random_state=999) # Take a subsample of the dataset for speed

We want to split right away - better not even look at summary stats of the test data, or even eyeball it.

In [3]:
imdb_train, imdb_test = train_test_split(imdb_df, random_state=123)

As a reminder, here is what we did last time:

In [4]:
X_train_imdb_raw = imdb_train['review']
y_train_imdb = imdb_train['label']

X_test_imdb_raw = imdb_test['review']
y_test_imdb = imdb_test['label']

In [5]:
vec = CountVectorizer(min_df=50, binary=True)

In [6]:
X_train_imdb = vec.fit_transform(X_train_imdb_raw)
X_test_imdb = vec.transform(X_test_imdb_raw);

In [10]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_imdb, y_train_imdb);

In [11]:
lr.score(X_train_imdb, y_train_imdb)

0.9833333333333333

In [12]:
lr.score(X_test_imdb, y_test_imdb)

0.8256

Last time, we avoided cross-validation. Why?

In [13]:
cross_val_score(lr, X_train_imdb, y_train_imdb)

array([0.82866667, 0.836     , 0.83733333, 0.83266667, 0.834     ])

- The code runs.
- But we have a problem... our good friend the Golden Rule.
- It is actually the exact same problem we fit/transformed the `CountVectorizer` before splitting.
- Remember, cross-validation involves splitting!!!

In [17]:
X_train_fold_1 = X_train_imdb[X_train_imdb.shape[0]//5:]
X_valid_fold_1 = X_train_imdb[:X_train_imdb.shape[0]//5]

- But wait, the validation part was transformed using a `CountVectorizer` that was fit on the training split.
- Just like last time, this is a Golden Rule violation.
- For example, the validation split "gets to be aware of" words that are only in the training split.

So what do we do here?

![](img/hmm.png)

Enter pipelines to the rescue!!

## Q&A

(Pause for Q&A)

<br><br><br><br>

## Pipelines (15 min)

- scikit-learn `Pipeline` can help us with this.

In [ ]:
from sklearn.pipeline import Pipeline

- This time we'll combine **the preprocessing and the model** with a `Pipeline`.

In [19]:
countvec = CountVectorizer(min_df=50, binary=True)
lr = LogisticRegression(max_iter=1000)

pipe = Pipeline([
    ('countvec', countvec),
    ('lr', lr)])

- Syntax: pass in a list of **steps**.
- The last step should be a model/classifier.
- All the earlier steps should be transformers.
  - Later in the course we'll see use cases for multiple rounds of transformers, here we only have one.

In [20]:
pipe.fit(X_train_imdb_raw, y_train_imdb);

- What is this doing?
- Note that I passed in the **raw** text data, not the vectorized word counts:

In [ ]:
X_train_imdb_raw

The pipeline is doing the following steps:

1. Fitting `CountVectorizer`.
2. Transforming the data using the fit `CountVectorizer`.
3. Fitting the `LogisticRegression` on the transformed data.

When we call `predict` (or `score`), we also feed in the raw data:

In [21]:
pipe.predict(X_test_imdb_raw)

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype=object)

Here is a schematic assuming you have two transformers:

<img src="img/pipeline.png" width="700">

[Source](https://amueller.github.io/COMS4995-s20/slides/aml-04-preprocessing/#18)

- One thing that is awesome here is that we can't make the mistakes we showed last time:
  - We call `fit` on the train split and `score` on the test split, it's clean.
  - We can't accidentally re-fit the preprocessor on the test data like we did last time.
  - It automatically makes sure the same transformations are applied to train and test.

And now, the moment of truth:

In [22]:
cross_val_score(pipe, X_train_imdb_raw, y_train_imdb)

array([0.82666667, 0.824     , 0.83133333, 0.83066667, 0.83533333])

- Remember what cross-validation does - it calls `fit` and `score`.
- Now we're calling `fit` on the pipeline, not just the logistic regression.
  - So **both the vectorizer and the logistic regression are refit again on each fold**.
  - This is what we want to avoid the Golden Rule violation!
  - Every validation score is unseen data with respect to the pipeline.

![](img/yay.png)

- BTW, the scores here aren't that different.
- I don't suspect it matters all that much here.
- But there could be cases where the effect is large.
- In this course I want you to build good habits that will serve you well going forward.

## Q&A

(Pause for Q&A)

<br><br><br><br>

## Break (5-10 min)

Please fill out the mid-course survey at https://ubc.ca1.qualtrics.com/jfe/form/SV_6tevNhMjZxRiQEl

<br><br><br><br>

## Hyperparameter optimization: grid search and random search (30 min)

#### Manual hyperparameter optimization

- We tried this a bit.
- Advantage: we may have some intuition about what might work.
  - E.g. if I'm massively overfitting, try decreasing `max_depth` or `C`.
- Disadvantage: it takes a lot of work.
- Disadvantage: in very complicated cases, our intuition might be worse than a data-driven approach.

#### Automated hyperparameter optimization

- Advantage: reduce human effort
- Advantage: less prone to error and improve reproducibility
- Advantage: data-driven approaches may be effective
- Disadvantage: may be hard to incorporate intuition
- Disadvantage: be careful about overfitting on the validation set.



There are two automated hyperparameter search methods in scikit-learn:

  - Exhaustive grid search: [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
  - Randomized hyperparameter optimization: [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
  
The "CV" stands for cross-validation; these searchers have cross-validation built right in.

In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Exhaustive grid search

- A user specifies a set of values for each hyperparameter. 
- The method considers "product" of the sets and then evaluates each combination one by one.    

Let's start the automated hyperparameter optimization.

In [24]:
param_grid = {
    "C" : [0.01, 1, 10, 100]
}

In [25]:
lr = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(lr, param_grid, verbose=1)

- Note that we can fix some hyperparameters and make others variable.
- `verbose=1` tells `GridSearchCV` to print some output while it's working.
  - This can be useful as this step sometimes takes a long time.

In [26]:
grid_search.fit(X_train_imdb, y_train_imdb);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   11.8s finished


Going back to Lecture 3, this is what it's doing:

```
for C in [0.01, 1, 10, 100]:
    for fold in folds:
        fit in training portion with the given C
        score on validation portion
    compute average score
pick hypers with best score
```

From here, we can extract the best hyperparameter values:

In [27]:
grid_search.best_params_

{'C': 0.01}

In [28]:
grid_search.best_score_

0.8474666666666668

We can extract the classifier inside like this:

In [29]:
grid_search.best_estimator_

LogisticRegression(C=0.01, max_iter=1000)

In [30]:
grid_search.best_estimator_.predict(X_test_imdb)

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype=object)

They also provide some "syntactic sugar" and allow you to call `predict` or `score` directly on the `GridSearchCV` object:

In [31]:
grid_search.predict(X_test_imdb) ## Does the same thing

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype=object)

By the way, by default it takes the best hyperparameters and refits on the entire training set - very nice!

In [33]:
# ?GridSearchCV

Note the `refit=True` to control this.

## Q&A

(Pause for Q&A)

<br><br><br><br>

- Ok, so this is all the syntax, but now we know we've been violating the Golden Rule because of the cross-validation.
- Furthermore, we may want to tune the hyperparameters of the `CountVectorizer` and the `LogisticRegression` together.
- Pipelines are perfect for this!!
- So let's do it again properly this time.

In [38]:
countvec = CountVectorizer(binary=True) # we should not set min_df here, it will be optimized
lr = LogisticRegression(max_iter=1000)  # we should not set C here, it will be optimized

pipe = Pipeline([
    ('countvec', countvec),
    ('lr', lr)])

In [39]:
param_grid = {
    "countvec__min_df" : [0, 10, 100],
    "lr__C" : [0.01, 1, 10, 100]
}

- Above: we have a nesting of transformers. 
- We can access the parameters of the "inner" objects by using `__` to go "deeper":
  - `countvec__min_df`: "the `min_df` of the `CountVectorizer` (of the pipeline)"
  - `lr__C`: "the `C` of the `LogisticRegression` (of the pipeline)"
- Later in the course we'll see even deeper nesting, like `preprocessor__numeric__imputer__strategy`.

So, now we pass int he `Pipeline` to `GridSearchCV`:

In [40]:
grid_search = GridSearchCV(pipe, param_grid, verbose=2, n_jobs=-1)

And we pass in the raw text because we're using a `Pipeline`:

In [41]:
grid_search.fit(X_train_imdb_raw, y_train_imdb);

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   44.0s finished


- Note the `n_jobs=-1` above.
- Hyperparameter optimization can be done _in parallel_ for each of the configurations.
- This is very useful when scaling up to large numbers of machines in the cloud.
- But even on my laptop there are 8 cores it can use, so that makes it a lot faster.

In [42]:
grid_search.best_params_

{'countvec__min_df': 0, 'lr__C': 1}

Heh, here we get back the defaults again. This happens surprisingly often - the defaults are well chosen!

- Note the number of candidates comes from the **product** of the number of options for each hyperparameter.
- And then the whole thing multiplied by the number of folds (default is 5).
- So, this number can get big really fast.

#### Problems with exhaustive grid search 

- Required number of models to evaluate grows exponentially with the dimensionally of the configuration space. 
- Exhaustive search may become infeasible fairly quickly. 

#### Randomized hyperparameter search

- Randomized hyperparameter optimization: [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- Samples configurations at random until certain budget (e.g., time) is exhausted.
- Advantage: you can choose how many runs you'll do.
- Advantage: you can restrict yourself less on what values you might try.
- Advantage: Adding parameters that do not influence the performance does not affect efficiency.
- Advantage: research shows this is generally a better idea than grid search, see image for intuition:

![](img/randomsearch_bergstra.png)

Source: [Bergstra and Bengio, Random Search for Hyper-Parameter Optimization, JMLR 2012](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).

- You don't know in advance which hyperparameters are important for your problem.
- But some of them might be unimportant.
- In the left figure, 6 of the 9 searches are useless because they are only varying the unimportant parameter.
- In the right figure, all 9 searches are useful.

## Q&A

(Pause for Q&A)

<br><br><br><br>

Back to syntax. We can have continue to have the parameters chosen from a list:

In [43]:
param_choices = {
    "countvec__min_df" : [0, 10, 100],
    "lr__C" : [0.01, 1, 10, 100]
}

- In which case it will randomly pick from each list for each run. 
- So, we can afford to have many more values this time since we're not going to try all of them:

In [44]:
param_choices = {
    "countvec__min_df" : np.arange(0,100),
    "lr__C" : 2.0**np.arange(-5,5)
}

In [47]:
2.0**np.arange(-5,5)

array([ 0.03125,  0.0625 ,  0.125  ,  0.25   ,  0.5    ,  1.     ,
        2.     ,  4.     ,  8.     , 16.     ])

- Note the exponential range for `C`. This is quite common.
- There is no point trying $C=\{1,2,3\ldots,100\}$ because $C=1,2,3$ are too similar to each other.
- Often we're trying to find an order of magnitude, e.g. $C=\{0.01,0.1,1,10,100\}$. 
- We can also write that is $C=\{10^{-2},10^{-1},10^0,10^1,10^2\}$. 
- Or, in other words, $C$ values to try are $10^n$ for $n=-2,-1,0,1,2$ which is basically what we have above.

Another thing we can do is give probability distributions to draw from:

In [48]:
import scipy.stats

In [ ]:
param_choices = {
    "countvec__min_df" : scipy.stats.randint(low=0, high=300),
    "lr__C" : scipy.stats.randint(low=0, high=300) # TODO: this is lame, pick a continuous prob dist
}

- This is a bit fancy. What's nice is that you can have it concentrate more on certain values by setting the distribution. 
- Let's go back to:

In [49]:
param_choices = {
    "countvec__min_df" : np.arange(0,100),
    "lr__C" : 2.0**np.arange(-5,5)
}

In [50]:
random_search = RandomizedSearchCV(pipe, param_choices,
                                   n_iter = 12, 
                                   verbose = 1,
                                   n_jobs = -1,
                                   random_state = 123)

- Note the `n_iter`, we didn't need this for `GridSearchCV`.
  - Larger `n_iter` will take longer but do more searching.
  - Remember you still need to multiply by number of folds!
- I also set `random_state` but you don't have it.

In [51]:
random_search.fit(X_train_imdb_raw, y_train_imdb);

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   33.4s finished


- Note: some hyperparameters significantly affect the training time!
- For example, setting `min_df=0` is going to be much slower than `min_df=50`.
  - Because the former results in way more columns.
  - That makes everything slower, including the logistic regression training.
  - You'll see this on hw3.

In [52]:
random_search.best_params_

{'lr__C': 0.0625, 'countvec__min_df': 13}

- Now we get something different! 
- What's the score?

In [53]:
random_search.best_score_

0.8605333333333333

In [54]:
grid_search.best_score_

0.8561333333333334

- So, they are very slightly different.
- Is that difference important?
- Do we BELIEVE that difference? How to figure this out?

<br><br>


- Some strategies:
  - We can try it out on the test set.
  - We can look at the sub-scores of the folds.
  - Try cross-validation with more folds.

But first:  

In [62]:
#??RandomizedSearchCV

In [60]:
random_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_lr__C', 'param_countvec__min_df', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [57]:
pd.DataFrame(random_search.cv_results_)[['mean_test_score', 'param_countvec__min_df', 'param_lr__C', 'mean_fit_time', 'rank_test_score']].set_index("rank_test_score").sort_index()

,mean_test_score,param_countvec__min_df,param_lr__C,mean_fit_time
rank_test_score,,,,
1,0.860533,13,0.0625,2.644730
2,0.860267,5,0.03125,3.331615
3,0.858533,22,0.0625,2.490565
4,0.853200,20,0.25,3.042938
5,0.853067,46,0.0625,3.052222
6,0.849200,63,0.125,2.913377
7,0.845600,19,1,3.478485
8,0.840400,13,8,4.030078
9,0.832000,58,1,3.735136


- Look at the timings, they can be quite interesting (try this on the homework!).
- And now, the test set:

In [64]:
grid_search.best_estimator_.score(X_test_imdb_raw, y_test_imdb)

0.8556

In [58]:
grid_search.score(X_test_imdb_raw, y_test_imdb)

0.8556

In [59]:
random_search.score(X_test_imdb_raw, y_test_imdb)

0.8544

## Q&A

(Pause for Q&A)

<br><br><br><br>

## Bayesian hyperparameter optimization (10 min)

- Both `GridSearchCV` and `RandomizedSearchCV` do each trial independently.
- What if you could learn from your experience, e.g. learn that `max_depth=3` is bad?
  - That could save time because you wouldn't try combinations involving `max_depth=3` in the future.
- We can do this with `scikit-optimize`, which is a completely different package from `scikit-learn`
- It uses a technique called "model-based optimization" and we'll specifically use "Bayesian optimization".
  - In short, it uses machine learning to predict what hyperparameters will be good.
  - Machine learning on machine learning!
- As it happens I did my PhD thesis on this topic.

In [65]:
from skopt import BayesSearchCV

- `BayesSearchCV` uses the same interface as `GridSearchCV` and `RandomSearchCV`.
- However, the way we specify the parameter distributions is slightly different.
- Here, we can just give the bounds as tuples.

In [66]:
bayes_opt = BayesSearchCV(
    pipe,
    {
        'countvec__min_df': (0, 300),   # This gets interpreted as a range
        'lr__C': (0.25, 0.5, 1, 2, 4, 8, 16, 32) # This gets interpreted as a list.
    },
    n_iter=10,
    cv=3,
    random_state=123,
    verbose=0,
    refit=True
)

In [67]:
%%time
bayes_opt.fit(X_train_imdb_raw, y_train_imdb);

CPU times: user 1min 1s, sys: 2.53 s, total: 1min 3s
Wall time: 55.3 s


BayesSearchCV(cv=3,
              estimator=Pipeline(steps=[('countvec',
                                         CountVectorizer(binary=True)),
                                        ('lr',
                                         LogisticRegression(max_iter=1000))]),
              n_iter=10, random_state=123,
              search_spaces={'countvec__min_df': (0, 300),
                             'lr__C': (0.25, 0.5, 1, 2, 4, 8, 16, 32)})

- It took longer than the other methods.
  - In reality there is some extra computation to do the "meta-ML".
  - This is not that significant here.
  - The overall time is dominated by the time of calling `fit`.
  - Another reason it look longer is because of the `n_jobs` issue (more on this below).

In [68]:
bayes_opt.best_params_

OrderedDict([('countvec__min_df', 8), ('lr__C', 32.0)])

In [69]:
bayes_opt.best_score_

0.844

- In theory, it should get even better as we increase `n_iter` (because it has more data to learn from).
- Checking the test score:

In [70]:
bayes_opt.score(X_test_imdb_raw, y_test_imdb)

0.84

And reproducing the previous test scores for comparison:

In [71]:
random_search.score(X_test_imdb_raw, y_test_imdb)

0.8544

In [72]:
grid_search.score(X_test_imdb_raw, y_test_imdb)

0.8556

- In this case, this didn't seem very effective. Sometimes it is. Especially with more hyperparameters and more trials.
- The more trials you do, the more it can learn from past experience.
- Should I always use this? Not necessarily. (Heh, well I guess this is one case where it wasn't great.)
- Disadvantage: requires installation.
- Disadvantage: when number of trials is large (e.g. hundreds), the meta-ML can actually get too slow.
- Disadvantage: harder parallelize the search because each trial depends on the previous ones.
  - Note `n_jobs` parameter for `GridSearchCV` and `RandomizedSearchCV`.  
  - `BayesSearchCV` also has this parameter.
  - It can definitely parallelize the folds.
  - The search will be less effective if it parallelizes further.
- I feel there's kind of a "sweet spot" of maybe ~10 continuous hyperparameters and ~100 trials where this tends to do really well.

- Can I generalize this to say `BayesSearchCV` > `RandomizedSearchCV` > `GridSearchCV`?
- Not quite. I'd say `RandomizedSearchCV` > `GridSearchCV` is pretty reasonable most of the time.
- But we should think a bit more carefully about `BayesSearchCV` for the above reasons.
- `RandomizedSearchCV` is often a reasonable choice.

## Q&A

(Pause for Q&A)

<br><br><br><br>